In [1]:
import sklearn_crfsuite
from sklearn_crfsuite import scorers
from sklearn_crfsuite import metrics

from nltk.tag.stanford import StanfordPOSTagger
from stanford_postagger.stanford_wrapper import StanfordPOSTagger as StanfordPOSTaggerWrapper

from sklearn.metrics import make_scorer
from sklearn.model_selection import cross_val_score

import scipy
from sklearn.grid_search import RandomizedSearchCV

/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/home/agus/anaconda3/envs/basic/lib/python3.6/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [2]:
%load_ext autoreload
%autoreload 2

# Read Dataset

In [3]:
f = open('datasets/conll2003/train.txt', 'r')
lines = f.readlines()
f.close()

del lines[0]
del lines[0]

dataset = []
sentence = []
for line in lines:
    splitter = line.strip().split(' ')
    if splitter[0] == '':
        continue
    elif (splitter[0] == '-DOCSTART-'):
        dataset.append(sentence)
        sentence = []
    else:
        token = splitter[0]
        tag = splitter[3]
        sentence.append((token, tag))

In [4]:
def convert_conlltxt2dataset(filename):
    f = open(filename, 'r')
    lines = f.readlines()
    f.close()
    
    del lines[0]
    del lines[0]
    
    dataset = []
    sentence = []
    for line in lines:
        splitter = line.strip().split(' ')
        if splitter[0] == '':
            continue
        elif (splitter[0] == '-DOCSTART-'):
            dataset.append(sentence)
            sentence = []
        else:
            token = splitter[0]
            tag = splitter[3]
            sentence.append((token, tag))
    return dataset

In [5]:
train_dataset = convert_conlltxt2dataset('datasets/conll2003/train.txt')
validation_dataset = convert_conlltxt2dataset('datasets/conll2003/valid.txt')
test_dataset = convert_conlltxt2dataset('datasets/conll2003/test.txt')

In [6]:
train_dataset[0][0:5]

[('EU', 'B-ORG'),
 ('rejects', 'O'),
 ('German', 'B-MISC'),
 ('call', 'O'),
 ('to', 'O')]

# Add Postag to Dataset

## Example

In [7]:
postagger = StanfordPOSTaggerWrapper()
postag = postagger.tag('+44 171')
postag

[('+44', 'CD'), ('171', 'CD')]

In [8]:
def add_postag2dataset(dataset):
    postagger = StanfordPOSTaggerWrapper()
    dataset_with_postag = []
    for sent in dataset:
        postagged_sent = []
        for index, (token, tag) in enumerate(sent):
            postagged_token = postagger.tag(token)
            postagged_sent.append((token, postagged_token[0][1], tag))
        dataset_with_postag.append(postagged_sent)
        
    return dataset_with_postag

postagged_train_dataset = add_postag2dataset(train_dataset)
postagged_validation_dataset = add_postag2dataset(validation_dataset)
postagged_test_dataset = add_postag2dataset(test_dataset)

# Delete Unused Dataset
del train_dataset
del validation_dataset
del test_dataset

In [9]:
postagged_train_dataset[0][0:5]

[('EU', 'NNP', 'B-ORG'),
 ('rejects', 'VBZ', 'O'),
 ('German', 'JJ', 'B-MISC'),
 ('call', 'NN', 'O'),
 ('to', 'TO', 'O')]

# Extract Feature

In [10]:
def word2features(sent, i):
    word = sent[i][0]
    postag = sent[i][1]

    # Ortographic Feature, Word, POSTag & N-Gram
    features = {
        'word.lower()': word.lower(),
        'word[-3:]': word[-3:],
        'word[-2:]': word[-2:],
        'word[:2]': word[:2],
        'word[:3]': word[:3],
        'word.istitle()': word.istitle(),
        'word.isdigit()': word.isdigit(),
        'word.isupper()': word.isupper(),
        'postag': postag,
        'postag[:2]': postag[:2]
    }
    
    # Position
    features.update({
        'pos_front': i,
        'pos_end': len(sent) - i
    })
    
    # Bag Of Words
    if i > 0:
        word1 = sent[i-1][0]
        postag1 = sent[i-1][1]
        features.update({
            '-1:word.lower()': word1.lower(),
            '-1:word.istitle()': word1.istitle(),
            '-1:word.isupper()': word1.isupper(),
            '-1:postag': postag1,
            '-1:postag[:2]': postag1[:2],
        })
    else:
        features['BOS'] = True
        
    if i < len(sent) - 1:
        word1 = sent[i+1][0]
        postag1 = sent[i+1][1]
        features.update({
            '+1:word.lower()': word1.lower(),
            '+1:word.istitle()': word1.istitle(),
            '+1:word.isupper()': word1.isupper(),
            '+1:postag': postag1,
            '+1:postag[:2]': postag1[:2],
        })
    else:
        features['EOS'] = True

    return features

def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]

def sent2postag(sent):
    return [postag for token, postag, label in sent]

def sent2labels(sent):
    return [label for token, postag, label in sent]

def sent2tokens(sent):
    return [token for token, postag, label in sent]

In [11]:
sent2features(postagged_train_dataset[0])[0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB'}

# Feature Extraction

In [12]:
X_train = [sent2features(sent) for sent in postagged_train_dataset]
y_train = [sent2labels(sent) for sent in postagged_train_dataset]

X_val = [sent2features(sent) for sent in postagged_validation_dataset]
y_val = [sent2labels(sent) for sent in postagged_validation_dataset]

X_test = [sent2features(sent) for sent in postagged_test_dataset]
y_test = [sent2labels(sent) for sent in postagged_test_dataset]

del postagged_train_dataset
del postagged_validation_dataset
del postagged_test_dataset

In [13]:
X_train[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB'}

# Load Word Embedding and Add to The Feature

In [14]:
def load_word_embedding_model(filename):
    f = open(filename, 'r', encoding='utf-8')
    lines = f.readlines()
    del lines[0]

    embedding_dict = {}
    counter = 0
    for line in lines:
        line = line.strip()
        arr = line.split(' ')
        word = arr[0].lower()
        vectors = [float(x) for x in arr[1:]]
        embedding_dict[word] = vectors
    return embedding_dict

def add_word_embedding2feature(embedding_dict, feature):
    VECTOR_SPACE_SIZE = 50
    for i, sentence in enumerate(feature):
        for j, token in enumerate(sentence):
            word = token['word.lower()']
            vector = []
            if word in embedding_dict:
                vector = embedding_dict[word]
            else:
                vector = [0 for i in range(VECTOR_SPACE_SIZE)]
            for k in range(len(vector)):
                wk = 'w{}'.format(k+1)
                feature[i][j][wk] = vector[k]
    return feature

from copy import deepcopy

In [15]:
cbow_embedding_dict = load_word_embedding_model('models/word_embedding/cbow_model.txt')
X_cbow = deepcopy(X_train)
X_cbow = add_word_embedding2feature(cbow_embedding_dict, X_cbow)

del cbow_embedding_dict

In [21]:
sg_embedding_dict = load_word_embedding_model('models/word_embedding/sg_model.txt')
X_sg = deepcopy(X_train)
X_sg = add_word_embedding2feature(sg_embedding_dict, X_sg)

del sg_embedding_dict

In [22]:
fasttext_embedding_dict = load_word_embedding_model('models/word_embedding/fasttext_model.txt')
X_fasttext = deepcopy(X_train)
X_fasttext = add_word_embedding2feature(fasttext_embedding_dict, X_fasttext)

del fasttext_embedding_dict

In [23]:
del X_train

In [24]:
X_cbow[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB',
 'w1': 0.018517746,
 'w2': 1.4631176,
 'w3': -0.18639173,
 'w4': 1.3069087,
 'w5': 0.21443413,
 'w6': 1.0161471,
 'w7': 0.20832239,
 'w8': 0.08281942,
 'w9': -0.8169725,
 'w10': 0.3771213,
 'w11': -0.8022201,
 'w12': 1.0028068,
 'w13': -0.22655097,
 'w14': 0.09523303,
 'w15': -0.18187046,
 'w16': -1.0263757,
 'w17': 0.686662,
 'w18': 0.24516387,
 'w19': 1.2970364,
 'w20': -0.47986302,
 'w21': -0.6222423,
 'w22': 0.13895805,
 'w23': 0.83969325,
 'w24': -0.031511858,
 'w25': 0.6807943,
 'w26': 0.661211,
 'w27': 1.7877787,
 'w28': -0.006412131,
 'w29': -0.23526877,
 'w30': 0.1048236,
 'w31': 0.177

In [25]:
X_sg[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB',
 'w1': 0.10116119,
 'w2': 0.0697031,
 'w3': -0.086072564,
 'w4': 0.8721925,
 'w5': 0.6003412,
 'w6': 0.25058785,
 'w7': 0.5479575,
 'w8': -0.16468725,
 'w9': -0.5230241,
 'w10': -0.24333891,
 'w11': -0.5888011,
 'w12': 0.85759246,
 'w13': 0.039961062,
 'w14': -0.0027838466,
 'w15': -0.7730575,
 'w16': -0.84855545,
 'w17': -0.56566066,
 'w18': -0.14627732,
 'w19': 0.6368246,
 'w20': -0.3295281,
 'w21': -0.78713435,
 'w22': -0.5496535,
 'w23': 0.49606988,
 'w24': -0.21009937,
 'w25': 0.75403863,
 'w26': 0.8362489,
 'w27': 1.6242998,
 'w28': -0.47526187,
 'w29': 0.419126,
 'w30': 0.15915819,
 'w31

In [26]:
X_fasttext[0][0]

{'word.lower()': 'eu',
 'word[-3:]': 'EU',
 'word[-2:]': 'EU',
 'word[:2]': 'EU',
 'word[:3]': 'EU',
 'word.istitle()': False,
 'word.isdigit()': False,
 'word.isupper()': True,
 'postag': 'NNP',
 'postag[:2]': 'NN',
 'pos_front': 0,
 'pos_end': 469,
 'BOS': True,
 '+1:word.lower()': 'rejects',
 '+1:word.istitle()': False,
 '+1:word.isupper()': False,
 '+1:postag': 'VBZ',
 '+1:postag[:2]': 'VB',
 'w1': -0.12108027,
 'w2': -0.10240047,
 'w3': 0.034586996,
 'w4': -0.043040045,
 'w5': -0.15763804,
 'w6': 0.14179787,
 'w7': -0.04570103,
 'w8': -0.016295906,
 'w9': -0.10652885,
 'w10': -0.041670088,
 'w11': -0.04865849,
 'w12': 0.067041956,
 'w13': -0.19511244,
 'w14': -0.059062067,
 'w15': 0.14742428,
 'w16': 0.032833103,
 'w17': 0.024178492,
 'w18': 0.03229901,
 'w19': -0.046403915,
 'w20': 0.011488709,
 'w21': 0.03376751,
 'w22': -0.041358087,
 'w23': 0.06666262,
 'w24': 0.02660803,
 'w25': 0.08017783,
 'w26': 0.07341458,
 'w27': 0.01881025,
 'w28': 0.046636917,
 'w29': 0.078622654,
 'w3

# Train Classifier Using Best Parameter

In [27]:
c1_ = 0.001262621084804322
c2_ = 0.07748342053200617

In [30]:
%%time
crf_cbow = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1_,
    c2=c1_,
    max_iterations=100,
    all_possible_transitions=True
)
crf_cbow.fit(X_cbow, y_train)

CPU times: user 1min 3s, sys: 793 ms, total: 1min 3s
Wall time: 1min 3s


In [31]:
%%time
crf_sg = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1_,
    c2=c1_,
    max_iterations=100,
    all_possible_transitions=True
)
crf_sg.fit(X_sg, y_train)

CPU times: user 1min 5s, sys: 318 ms, total: 1min 5s
Wall time: 1min 5s


In [32]:
%%time
crf_fasttext = sklearn_crfsuite.CRF(
    algorithm='lbfgs',
    c1=c1_,
    c2=c1_,
    max_iterations=100,
    all_possible_transitions=True
)
crf_fasttext.fit(X_fasttext, y_train)

CPU times: user 1min 5s, sys: 185 ms, total: 1min 5s
Wall time: 1min 5s


# Save Model

In [33]:
import pickle

In [34]:
filename_cbow = 'temp_crf_cbow.sav'
pickle.dump(crf_cbow, open(filename_cbow, 'wb'))

filename_sg = 'temp_crf_sg.sav'
pickle.dump(crf_sg, open(filename_sg, 'wb'))

filename_fasttext = 'temp_crf_fasttext.sav'
pickle.dump(crf_fasttext, open(filename_fasttext, 'wb'))